In [1]:
from pymongo import MongoClient
import pandas as pd
import requests
import src.functions1 as f1

In [2]:
# Activating and connecting to MongoDB database

client = MongoClient("localhost:27017")
db = client.get_database('Crunchbase')
c = db.get_collection('Crunchbase')

In [3]:
# Comprobating the existence of web companies founded after 2004, with 10-100 employees, in the selected cities:

proj = {"_id": 0, "name": 1, "offices.city":1, "number_of_employees": 1, "offices.latitude" : 1, "offices.longitude" : 1}

condition1 = {"category_code": "web"}
condition2 = {"$or": [{"offices.city": "Madrid"}, {"offices.city":"Cape Town"}, {"offices.city": "Miami"}]}
condition3 = {"number_of_employees": {"$lt":100, "$gt":10}}
condition4 = {"founded_year": {"$gt": 2005}}

companies = list(c.find({"$and": [condition1,condition2,condition3,condition4]}, proj))
df = pd.DataFrame(companies)
df["offices"] = df.offices.astype(str)
df = df.drop_duplicates()
df

,name,number_of_employees,offices
0,Yola,70,"[{'city': 'San Francisco', 'latitude': 37.7911..."
1,Call Me Later,25,"[{'city': 'Miami', 'latitude': 25.916982, 'lon..."
3,Zankyou,15,"[{'city': 'Madrid', 'latitude': 40.6070897, 'l..."


### Mongo Conclusion:

##### From Madrid, Miami and Capetown (my three favorite cities of all those I have visited, because of their similarity with the Spanish culture), I have check the feasibility of moving my new company there. I look at the criteria of: 

- Young company (founded after 2005).
- SME (less than fifty employees, and more than 10)
- Companies in the "web" sector

### 🚀All three options are viable‼ 🚀

--------------------------------------------------------------------------------------

In [4]:
# Requesting the location to geocode API (https://geocode.xyz) and creating a df

madrid = "madrid"
cape_town = "cape town"
miami = "miami"
df_madrid = pd.DataFrame(requests.get(f"https://geocode.xyz/{madrid}?json=1").json())
df_cape = pd.DataFrame(requests.get(f"https://geocode.xyz/{cape_town}?json=1").json())
df_miami = pd.DataFrame(requests.get(f"https://geocode.xyz/{miami}?json=1").json())
df_geocode = f1.df_final(df_madrid, df_cape, df_miami)
df_geocode

,longt,latt
city,,
Madrid,-3.67930,40.42955
Cape Town,18.46654,-33.99268
Miami,-80.28966,25.76849


In [5]:
# Saving the df in the correct folder

df_geocode.to_csv('../Maps_df/situation.csv')